In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
data_Kepler = pd.read_csv("Resources/cleaned_Kepler.csv")
data_Kepler.head()

,Unnamed: 0,koi_pdisposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,CANDIDATE,0,0,0,0,9.488036,170.538750,0.146,2.95750,...,793.0,93.59,35.8,1.0,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,1,CANDIDATE,0,0,0,0,54.418383,162.513840,0.586,4.50700,...,443.0,9.11,25.8,2.0,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,2,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,...,638.0,39.30,76.3,1.0,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,3,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,...,1395.0,891.96,505.6,1.0,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,4,CANDIDATE,0,0,0,0,2.525592,171.595550,0.701,1.65450,...,1406.0,926.16,40.9,1.0,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [3]:
X = data_Kepler.drop(columns=["koi_pdisposition", "Unnamed: 0"], axis=1)
y = data_Kepler["koi_pdisposition"]
print(X.shape, y.shape)

(8945, 20) (8945,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [5]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical
y_test_categorical

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [11]:
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=20))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [12]:
deep_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 12,402
Trainable params: 12,402
Non-trainable params: 0
_________________________________________________________________


In [14]:
deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
210/210 - 0s - loss: 0.1268 - accuracy: 0.9823
Epoch 2/50
210/210 - 0s - loss: 0.0318 - accuracy: 0.9934
Epoch 3/50
210/210 - 0s - loss: 0.0299 - accuracy: 0.9936
Epoch 4/50
210/210 - 0s - loss: 0.0287 - accuracy: 0.9934
Epoch 5/50
210/210 - 0s - loss: 0.0288 - accuracy: 0.9936
Epoch 6/50
210/210 - 0s - loss: 0.0291 - accuracy: 0.9936
Epoch 7/50
210/210 - 0s - loss: 0.0269 - accuracy: 0.9936
Epoch 8/50
210/210 - 0s - loss: 0.0265 - accuracy: 0.9936
Epoch 9/50
210/210 - 0s - loss: 0.0264 - accuracy: 0.9936
Epoch 10/50
210/210 - 0s - loss: 0.0252 - accuracy: 0.9933
Epoch 11/50
210/210 - 0s - loss: 0.0265 - accuracy: 0.9934
Epoch 12/50
210/210 - 0s - loss: 0.0261 - accuracy: 0.9936
Epoch 13/50
210/210 - 0s - loss: 0.0261 - accuracy: 0.9934
Epoch 14/50
210/210 - 0s - loss: 0.0254 - accuracy: 0.9936
Epoch 15/50
210/210 - 0s - loss: 0.0251 - accuracy: 0.9936
Epoch 16/50
210/210 - 0s - loss: 0.0247 - accuracy: 0.9939
Epoch 17/50
210/210 - 0s - loss: 0.0246 - accuracy: 0.9934
Epoch 

In [15]:
encoded_predictions = deep_model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

D:\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [16]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['CANDIDATE' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'CANDIDATE' 'FALSE POSITIVE']
Actual Labels: ['CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE']


In [17]:
deep_model_loss, deep_model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}")

70/70 - 0s - loss: 0.0420 - accuracy: 0.9928
Loss: 0.04203719273209572, Accuracy: 0.9928475618362427
